# Colab Notebook for Cellpose image restoration

Author: Dr.Yusuke Kimata (Ueda G)<br>
ref: Stringer, C., Pachitariu, M. Cellpose3: one-click image restoration for improved cellular segmentation. Nat Methods 22, 592–599 (2025). https://doi.org/10.1038/s41592-025-02595-5<br>
<br>
Input File Requirements : tiff stack, located in `/content/drive/MyDrive/Colab Notebooks/Cellpose/input`


## I. Preparation

In [1]:
# ============================================================
# 0) Environment setup (run once per Colab runtime) ───────────
# ============================================================
!pip install -q cellpose==3.1.1.2 tifffile ipywidgets scikit-image opencv-python-headless

import torch, os, subprocess
print("PyTorch detects GPU:", torch.cuda.is_available())
!nvidia-smi  # GPU model (optional)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Check Cellpose version
from cellpose.version import version_str
print("string:", version_str)

string: 
cellpose version: 	3.1.1.2 
platform:       	linux 
python version: 	3.11.13 
torch version:  	2.6.0+cu124


In [3]:
# ============================================================
# 1) Load a 3-D / 4-D TIFF stack  ─────────────────────────────
#    Expected axis order: (Z, Y, X) or (Z, C, Y, X)
# ============================================================
import tifffile as tif
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/

import os
HOME = os.getcwd()
print("HOME:", HOME)

%cd {HOME}/Colab Notebooks/Cellpose


Mounted at /content/drive
/content/drive/MyDrive
HOME: /content/drive/MyDrive
/content/drive/MyDrive/Colab Notebooks/Cellpose


In [4]:
img_name = '1_original' # <-- change to your own file
tiff_path = './input/' + img_name + '.tif'
stack = tif.imread(tiff_path)
print("Input shape:", stack.shape)

import ipywidgets as widgets

z_mid  = stack.shape[0] // 2

def show_raw(z):
    """Interactive viewer for raw slices."""
    plt.figure(figsize=(4, 4))
    plt.imshow(stack[z], cmap='gray')
    plt.title(f'Raw slice  z={z}')
    plt.axis('off')
    plt.show()

widgets.interact(
    show_raw,
    z=widgets.IntSlider(min=0, max=stack.shape[0]-1,
                        step=1, value=z_mid,
                        description='Z-slice',
                        continuous_update=False)
);

Input shape: (297, 512, 512)


interactive(children=(IntSlider(value=148, continuous_update=False, description='Z-slice', max=296), Output())…

## II. Denoise

In [5]:
# ============================================================
# 2) Axis semantics & Cellpose model  ────────────────────────
# ============================================================
z_axis       = 0          # index of Z
channel_axis = None       # None if single-channel, else e.g. 1 for (Z,C,Y,X)

from cellpose import denoise
dn = denoise.DenoiseModel(model_type="denoise_cyto3", gpu=True) # "denoise_cyto3" or "denoise_nuclei"

print("Model path:", dn.pretrained_model)

100%|██████████| 25.3M/25.3M [00:00<00:00, 63.4MB/s]


Model path: /root/.cellpose/models/denoise_cyto3


In [6]:
# ============================================================
# 3) Run 3-D Cellpose denoising  ──────────────────────────
# ============================================================
imgs_dn = dn.eval(stack, channels=None, diameter=10.)
print("Denoise finished")

Denoise finished


In [7]:
# ============================================================
# 4) Show result (raw + denoised) ─────────────────────────────
# ============================================================

def show_denoised(z):
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))

    # Raw image
    ax[0].imshow(stack[z], cmap='viridis')
    ax[0].set_title(f'Raw   z={z}')
    ax[0].axis('off')

    # Denoised image
    ax[1].imshow(imgs_dn[z], cmap='viridis')
    ax[1].set_title('Denoised')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

widgets.interact(
    show_denoised,
    z=widgets.IntSlider(
        min=0,
        max=stack.shape[0] - 1,
        step=1,
        value=stack.shape[0] // 2,
        description='Z-slice',
        continuous_update=False
    )
);

interactive(children=(IntSlider(value=148, continuous_update=False, description='Z-slice', max=296), Output())…

In [8]:
# ============================================================
# 5) Save results as BigTIFF  ──────────────────────────────────
# ============================================================
out_path = './output/' + img_name + '_denoised.tif'

vmin, vmax = imgs_dn.min(), imgs_dn.max()
imgs_dn_16bit = ((imgs_dn - vmin) / (vmax - vmin) * 65535).clip(0, 65535).astype(np.uint16)
tif.imwrite(out_path, imgs_dn_16bit, bigtiff=True)
print("Denoised 16-bit stack written to", out_path)

Denoised 16-bit stack written to ./output/1_original_denoised.tif


## III. Deblurring

In [9]:
# ============================================================
# 2) Axis semantics & Cellpose model  ────────────────────────
# ============================================================
z_axis       = 0          # index of Z
channel_axis = None       # None if single-channel, else e.g. 1 for (Z,C,Y,X)

from cellpose import denoise
db = denoise.DenoiseModel(model_type="deblur_cyto3", gpu=True) # "deblur_cyto3" or "deblur_nuclei"

print("Model path:", db.pretrained_model)

100%|██████████| 25.3M/25.3M [00:00<00:00, 28.4MB/s]


Model path: /root/.cellpose/models/deblur_cyto3


In [10]:
# ============================================================
# 3) Run 3-D Cellpose deblurring  ──────────────────────────
# ============================================================
imgs_db = db.eval(stack, channels=None, diameter=10.)
print("Deblur finished")

Deblur finished


In [11]:
# ============================================================
# 4) Show result (raw + deblured) ─────────────────────────────
# ============================================================

def show_deblured(z):
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))

    # Raw image
    ax[0].imshow(stack[z], cmap='viridis')
    ax[0].set_title(f'Raw   z={z}')
    ax[0].axis('off')

    # Deblured image
    ax[1].imshow(imgs_db[z], cmap='viridis')
    ax[1].set_title('Deblured')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

widgets.interact(
    show_deblured,
    z=widgets.IntSlider(
        min=0,
        max=stack.shape[0] - 1,
        step=1,
        value=stack.shape[0] // 2,
        description='Z-slice',
        continuous_update=False
    )
);

interactive(children=(IntSlider(value=148, continuous_update=False, description='Z-slice', max=296), Output())…

In [12]:
# ============================================================
# 5) Save results as BigTIFF  ──────────────────────────────────
# ============================================================
out_path = './output/' + img_name + '_deblured.tif'

vmin, vmax = imgs_db.min(), imgs_db.max()
imgs_db_16bit = ((imgs_db - vmin) / (vmax - vmin) * 65535).clip(0, 65535).astype(np.uint16)
tif.imwrite(out_path, imgs_db_16bit, bigtiff=True)
print("Deblured 16-bit stack written to", out_path)

Deblured 16-bit stack written to ./output/1_original_deblured.tif


## IV. Upsampling

In [13]:
# ============================================================
# 2) Axis semantics & Cellpose model  ────────────────────────
# ============================================================
z_axis       = 0          # index of Z
channel_axis = None       # None if single-channel, else e.g. 1 for (Z,C,Y,X)

from cellpose import denoise
us = denoise.DenoiseModel(model_type="upsample_cyto3", gpu=True) # "upsample_cyto3" or "upsample_nuclei"

print("Model path:", us.pretrained_model)

100%|██████████| 25.4M/25.4M [00:00<00:00, 74.6MB/s]


Model path: /root/.cellpose/models/upsample_cyto3


In [14]:
# ============================================================
# 3) Run 3-D Cellpose upsampling  ──────────────────────────
# ============================================================
imgs_us = us.eval(stack, channels=None, diameter=30.)
print("Upsampling finished")

Upsampling finished


In [15]:
# ============================================================
# 4) Show result (raw + upsampled) ─────────────────────────────
# ============================================================

def show_upsampled(z):
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))

    # Raw image
    ax[0].imshow(stack[z], cmap='viridis')
    ax[0].set_title(f'Raw   z={z}')
    ax[0].axis('off')

    # Upsampled image
    ax[1].imshow(imgs_us[z], cmap='viridis')
    ax[1].set_title('Upsampled')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

widgets.interact(
    show_upsampled,
    z=widgets.IntSlider(
        min=0,
        max=stack.shape[0] - 1,
        step=1,
        value=stack.shape[0] // 2,
        description='Z-slice',
        continuous_update=False
    )
);

interactive(children=(IntSlider(value=148, continuous_update=False, description='Z-slice', max=296), Output())…

In [16]:
# ============================================================
# 5) Save results as BigTIFF  ──────────────────────────────────
# ============================================================
out_path = './output/' + img_name + '_upsampled.tif'

vmin, vmax = imgs_us.min(), imgs_us.max()
imgs_us_16bit = ((imgs_us - vmin) / (vmax - vmin) * 65535).clip(0, 65535).astype(np.uint16)
tif.imwrite(out_path, imgs_us_16bit, bigtiff=True)
print("Upsampled 16-bit stack written to", out_path)

Upsampled 16-bit stack written to ./output/1_original_upsampled.tif


## V. Oneclick

In [17]:
# ============================================================
# 2) Axis semantics & Cellpose model  ────────────────────────
# ============================================================
z_axis       = 0          # index of Z
channel_axis = None       # None if single-channel, else e.g. 1 for (Z,C,Y,X)

from cellpose import denoise
oc = denoise.DenoiseModel(model_type="oneclick_cyto3", gpu=True) # "oneclick_cyto3" or "oneclick_nuclei"

print("Model path:", oc.pretrained_model)

100%|██████████| 25.4M/25.4M [00:00<00:00, 74.3MB/s]


Model path: /root/.cellpose/models/oneclick_cyto3


In [ ]:
# ============================================================
# 3) Run 3-D Cellpose restoring  ──────────────────────────
# ============================================================
imgs_oc = oc.eval(stack, channels=None, diameter=30.)
print("Restore finished")

In [ ]:
# ============================================================
# 4) Show result (raw + restored) ─────────────────────────────
# ============================================================

def show_restored(z):
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))

    # Raw image
    ax[0].imshow(stack[z], cmap='viridis')
    ax[0].set_title(f'Raw   z={z}')
    ax[0].axis('off')

    # Restored image
    ax[1].imshow(imgs_oc[z], cmap='viridis')
    ax[1].set_title('Restored')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

widgets.interact(
    show_restored,
    z=widgets.IntSlider(
        min=0,
        max=stack.shape[0] - 1,
        step=1,
        value=stack.shape[0] // 2,
        description='Z-slice',
        continuous_update=False
    )
);

In [ ]:
# ============================================================
# 5) Save results as BigTIFF  ──────────────────────────────────
# ============================================================
out_path = './output/' + img_name + '_restored.tif'

vmin, vmax = imgs_oc.min(), imgs_oc.max()
imgs_oc_16bit = ((imgs_oc - vmin) / (vmax - vmin) * 65535).clip(0, 65535).astype(np.uint16)
tif.imwrite(out_path, imgs_oc_16bit, bigtiff=True)
print("Restored 16-bit stack written to", out_path)